In [27]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
df = pd.read_csv('movie_dataset.csv')
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [29]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [30]:
features = ['genres', 'keywords', 'popularity', 'cast', 'director']
data = pd.DataFrame(df['title'])
for f in features:
    temp = pd.DataFrame(df[f])
    data = pd.concat([data,temp],axis=1)

In [31]:
data.head()

,title,genres,keywords,popularity,cast,director
0,Avatar,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,150.437577,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,139.082615,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Spectre,Action Adventure Crime,spy based on novel secret agent sequel mi6,107.376788,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,The Dark Knight Rises,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,112.312950,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,John Carter,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,43.926995,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [32]:
len(data[data.isna().any(axis=1)]) / len(data) * 100

8.869456589631481

In [33]:
# so we cant get rid on nan, put empty string there
data.fillna('',inplace=True)

In [34]:
len(data[data.isna().any(axis=1)])

0

In [35]:
data.shape

(4803, 6)

In [36]:
dataX = data.drop(['title','popularity'],axis=1)

In [37]:
dataX["combined_features"] = dataX['genres'] + " " + dataX['keywords'] + " " + dataX['cast'] + " " + dataX['director']

In [38]:
dataX['combined_features'].head()

0    Action Adventure Fantasy Science Fiction cultu...
1    Adventure Fantasy Action ocean drug abuse exot...
2    Action Adventure Crime spy based on novel secr...
3    Action Crime Drama Thriller dc comics crime fi...
4    Action Adventure Science Fiction based on nove...
Name: combined_features, dtype: object

In [39]:
cv = CountVectorizer()  

In [40]:
cv_mat = cv.fit_transform(dataX['combined_features']).toarray()

In [41]:
cv_mat

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [42]:
similarity = cosine_similarity(cv_mat)

In [60]:
#prompt user for input movie
def output(title):
    movie = title
    i = data[data['title'] == movie].index
    i = i[0]
    indexed_sim = list(enumerate(similarity[i]))
    sorted_sim = sorted(indexed_sim, key=lambda x:x[1], reverse=True)
    req_sim = sorted_sim[1:11]
    for i,_ in req_sim:
        print(data['title'][i])

In [61]:
output('Avatar')

Guardians of the Galaxy
Aliens
Star Wars: Clone Wars: Volume 1
Star Trek Into Darkness
Star Trek Beyond
Alien
Lockout
Jason X
The Helix... Loaded
Moonraker


In [62]:
output('Spectre')

Skyfall
Quantum of Solace
The Girl with the Dragon Tattoo
The Hunger Games: Catching Fire
Johnny English Reborn
One for the Money
The Adventurer: The Curse of the Midas Box
Nancy Drew
Diamonds Are Forever
The Other Side of Heaven


In [63]:
output('John Carter')

The Host
Heaven is for Real
Beastmaster 2: Through the Portal of Time
The Helix... Loaded
Transformers
The Martian
The Hunger Games: Catching Fire
Ender's Game
Divergent
Damnation Alley
